In [1]:
import pyphen
import string
import re
import collections

import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, Embedding, LSTM
import keras

import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(42)

Using TensorFlow backend.


In [2]:
%load_ext pep8_magic

In [2]:
hun_chars = 'aábcdeéfghiíjklmnoóöőpqrstuúüűvwxyz' + '^$'  # ^,$


def hyph_tags(word, hypher=pyphen.Pyphen(lang='hu_HU'), aslist=False):
    """Hyphenating classification of the characters in the word.
    {B(egin),M(iddle),E(nd),S(ingle)}"""
    if (len(word) == 0):
        raise IndexError("0 length word")
    ret = list('M' * len(word))
    ret[0] = 'B'
    ret[-1] = 'E'
    for i in hypher.positions(word):
        ret[i] = 'B'
        if(ret[i-1] == 'B'):
            ret[i-1] = 'S'
        else:
            ret[i-1] = 'E'
    if (aslist):
        return ret
    return "".join(ret)


def hyph_tags_4to2(word, aslist=False):
    """{B,M,E,S} to {B, M}"""
    ret = list(word)
    for i in range(len(ret)):
        if ret[i] == 'S':
            ret[i] = 'B'
        if ret[i] != 'B':
            ret[i] = 'M'
    if(aslist):
        return ret
    return "".join(ret)


def same_char_num(word, hypher=pyphen.Pyphen(lang='hu_HU')):
    """Return true if the hyphenated word has as many chars as the original"""
    return len(hypher.inserted(word)) == len(word)+len(hypher.positions(word))


def cleaning(data):
    """Text cleaning:
        lower the letters
        punctuation, digits ellimination"""
    formated_data = data.lower()
    formated_data = re.sub('['+string.punctuation+']', '', formated_data)
    formated_data = re.sub('['+string.digits+']', '', formated_data)
    return formated_data


# onehot: {'B','M','E','S'}
def one_hot_encode(char, dictionary='BMES'):
    ret = [0]*len(dictionary)
    if char in dictionary:
        ret[dictionary.find(char)] = 1
        return ret
    raise ValueError('Value out of dictionary range: '+char)


def unison_shuffled_copies(a, b):
    """Randomize 2 same length array in the same permutation"""
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]


def one_hot_decode(arr, dictionary='BMES'):
    assert len(arr) == len(dictionary)
    i = np.nonzero(arr)[0][0]
    return dictionary[i]


def generate_network_data(data, ret_input=[], ret_output=[],
                          length=2, length_after=0,
                          start_char='^', end_char='$',
                          chars=hun_chars, tag_chars='BMES'):
    """from [word,hyph_class(word) to length-long input-output data"""
    word = data[0]
    word_plus = start_char*(length-length_after-1)+word+end_char*length_after
    hyph_word = data[1]
    for i in range(0, len(word)):
        input_next_iter = []
        for c in word_plus[i:i+length]:
            input_next_iter.append(one_hot_encode(c, chars))
        output_next_iter = one_hot_encode(hyph_word[i], tag_chars)
        ret_input.append(input_next_iter)
        ret_output.append(output_next_iter)
    return

def generate_network_words(data, padding=None, start_char='^',
                           end_char='$', chars=hun_chars,
                           tag_chars='BMES', tag_default=-1):
    """One-hot [word, hyph_class(word)]->[[[010],[010]],[[01],[01]]]
    padding to fixed size, if not null"""
    ret_input=[]
    ret_output=[]
    
    word = data[0]
    hyph_word = data[1]
    if padding != None:
        if len(word)>padding:
            raise IndexError("The word is longer than the fixed size")
        else:
            word = word + (padding-len(word))*end_char
            hyph_word = hyph_word + (padding-len(hyph_word)) * tag_chars[tag_default]
    for i in range(0,len(word)):
        input_next_iter = one_hot_encode(word[i],chars)
        output_next_iter = one_hot_encode(hyph_word[i], tag_chars)
        ret_input.append(input_next_iter)
        ret_output.append(output_next_iter)
    return ret_input, ret_output
    
def hyph_tupples(data, hypher=pyphen.Pyphen(lang='hu_HU'),
                tag_chars='BM'):
    """[words] -> [words, hyph_words]"""
    word_list = []
    c_all = 0
    c_same_char_num = 0
    for next_word in data:
        c_all += 1
        if(len(next_word) != 0 and same_char_num(next_word, hypher)):
            c_same_char_num += 1
            if(len(tag_chars) == 2):
                word_list.append([next_word,
                                  hyph_tags_4to2(hyph_tags(next_word))])
            else:
                word_list.append([next_word, hyph_tags(next_word)])
    return word_list, c_all, c_same_char_num

def tupple_to_train(word_list, window_length, length_after,
                 tag_chars='BM'):
    """[words, hyph_words] -> in[0,1,0...], out[0,1,0...]"""
    data_in = []
    data_out = []
    wrong_word = 0
    for word in word_list:
        try:
            generate_network_data(word, data_in, data_out,
                                  window_length, tag_chars=tag_chars,
                                  length_after=length_after)
        except ValueError:
            wrong_word += 1
    return data_in, data_out, wrong_word

def bigram_counter_from_file(filename):
    """creates bigram counter from file"""
    with open(filename) as f:
        word_list = []
        for words in f:
            words = words.strip()
            words = words.split()
            for w in words:
                w = cleaning(w)
                if len(w)>0:
                    word_list.append(w)

    bigram_counter = collections.Counter()
    for word in word_list:
        for i in range(2,len(word)):
            bigram_counter[word[i-2:i]] += 1
    return bigram_counter

def bigrams_in_word(word, bigram_counter, mc=100):
    bigrams = np.array(bigram_counter.most_common(mc))[:,0]
    w_bc = len(word)-1
    if w_bc<1:
        return 1.0
    w_bf = 0
    for i in range(2,len(word)):
        if word[i-2:i] in bigrams:
            w_bf +=1
    return w_bf/w_bc

def bigram_selector(word, bigram_counters,threshold=0.2, mc=100):
    """Choose the language of the word"""
    lang_likes = np.zeros(len(bigram_counters)+1)
    for i in range(0,len(bigram_counters)):
        lang_likes[i] = bigrams_in_word(word, bigram_counters[i], mc)
    lang_likes_max = np.argmax(lang_likes)
    
    for i in range(0,len(bigram_counters)):
        if i!=lang_likes_max:
            if lang_likes[lang_likes_max]-lang_likes[i]<=threshold:
                return len(bigram_counters)
    return lang_likes_max

# Data import

In [3]:
def data_reader(file, tail_cut=100000,
                lang_selector = False, lang_thr=0.6,
                lang_file_en='../wikipedia/angol/ossz_angol',
                lang_file_hu='../wikipedia/magyar/ossz_magyar'):
    """Read data from file"""

    if lang_selector:
        bigram_counter_en = bigram_counter_from_file(lang_file_en)
        bigram_counter_hu = bigram_counter_from_file(lang_file_hu)
        out_en_words = 0
    
    tail_cut_ptest_words = tail_cut + 500

    counter_hu_data = collections.Counter()
    with open(file, 'r',
              errors='ignore', encoding='latin2') as f:
        i = 0
        for line in f:
            i = i+1
            words = line.split()
            if len(words) > 1:
                if(words[1].isdigit()):
                    cword = cleaning(words[0])
                    if lang_selector:
                        lang = bigram_selector(cword,
                                            [bigram_counter_hu,
                                             bigram_counter_en],
                                            lang_thr)
                        if (lang!=1):
                            counter_hu_data[cword] += int(words[1])
                        else:
                            out_en_words +=1
                    else:
                        counter_hu_data[cword] += int(words[1])
            if i > tail_cut_ptest_words:
                break
    if lang_selector:
        print("Throwed english words: ", out_en_words)
    return counter_hu_data

In [4]:
def train_data_generator(data_counter, window_length, length_after,
                         tag_chars='BM', tail_cut=100000,
                         valid_rate=0.2, test_rate=0.1):
    """Generate training data from counter data
    unique words -> characters -> randomize -> cut"""

    data_list = np.array(data_counter.most_common(tail_cut))[:,0]
    word_list, c_all, c_same_char_num = hyph_tupples(data_list,
                                                    tag_chars=tag_chars)
    print('Data read successfully')
    print('non-standard hyphenation:')
    print(c_same_char_num, c_all, c_same_char_num/c_all)

    # Generate network data
    data_in = []
    data_out = []
    wrong_word = 0
    data_in, data_out, wrong_word = tupple_to_train(word_list,
                                                    window_length,
                                                    length_after,
                                                    tag_chars=tag_chars)
    print('Data len: ', len(data_in))
    print('Words with unrecognized caracter: ', wrong_word)

    data_len = len(data_in)

    data_in = np.array(data_in, dtype='float32')
    data_out = np.array(data_out, dtype='float32')
    data_in, data_out = unison_shuffled_copies(data_in, data_out)
    tests_input = data_in[0:int(data_len*test_rate)]
    tests_target = data_out[0:int(data_len*test_rate)]
    valid_input = data_in[int(data_len*test_rate):
                          int(data_len*(test_rate+valid_rate))]
    valid_target = data_out[int(data_len*test_rate):
                            int(data_len*(test_rate+valid_rate))]
    train_input = data_in[int(data_len*(test_rate+valid_rate)):]
    train_target = data_out[int(data_len*(test_rate+valid_rate)):]

    print('Training data size:', np.shape(train_input), np.shape(train_target))
    print('Validation data size:', np.shape(valid_input),
          np.shape(valid_target))
    print('Test data size:', np.shape(tests_input), np.shape(tests_target))

    train_input_flatten = np.reshape(
        train_input, (len(train_input), (window_length)*len(hun_chars)))
    valid_input_flatten = np.reshape(
        valid_input, (len(valid_input), (window_length)*len(hun_chars)))
    tests_input_flatten = np.reshape(
        tests_input, (len(tests_input), (window_length)*len(hun_chars)))
    print('Network data generated successfully')

    return [train_input_flatten, train_target,
            valid_input_flatten, valid_target,
            tests_input_flatten, tests_target]


def train_data_generator_uwords(data_counter, window_length, length_after,
                                tag_chars='BM', tail_cut=100000,
                                valid_rate=0.2, test_rate=0.1):
    """Generate training data from counter data
        unique words -> randomize -> cut -> characters"""
    data_list = np.array(data_counter.most_common(tail_cut))[:,0]
    np.random.shuffle(data_list)
    data_len = len(data_list)
    tests_data = data_list[0:int(data_len*test_rate)]
    valid_data = data_list[int(data_len*test_rate):
                           int(data_len*(test_rate+valid_rate))]
    train_data = data_list[int(data_len*(test_rate+valid_rate)):]
    
    c_all = 0
    c_same_char_num = 0
    tests_list, c_all_p, c_same_char_num_p = hyph_tupples(tests_data,
                                                          tag_chars=tag_chars)
    c_all += c_all_p
    c_same_char_num += c_same_char_num_p
    valid_list, c_all_p, c_same_char_num_p = hyph_tupples(valid_data,
                                                          tag_chars=tag_chars)
    c_all += c_all_p
    c_same_char_num += c_same_char_num_p
    train_list, c_all_p, c_same_char_num_p = hyph_tupples(train_data,
                                                          tag_chars=tag_chars)
    c_all += c_all_p
    c_same_char_num += c_same_char_num_p
    
    print('Data read successfully')
    print('non-standard hyphenation:')
    print(c_same_char_num, c_all, c_same_char_num/c_all)
    
    wrong_word = 0
    tests_input, tests_target, wrong_w_p = tupple_to_train(tests_list,
                                                           window_length,
                                                           length_after,
                                                           tag_chars=tag_chars)
    wrong_word += wrong_w_p
    valid_input, valid_target, wrong_w_p = tupple_to_train(valid_list,
                                                           window_length,
                                                           length_after,
                                                           tag_chars=tag_chars)
    wrong_word += wrong_w_p
    train_input, train_target, wrong_w_p = tupple_to_train(train_list,
                                                           window_length,
                                                           length_after,
                                                           tag_chars=tag_chars)
    wrong_word += wrong_w_p
    print('Words with unrecognized caracter: ', wrong_word)

    print('Training data size:', np.shape(train_input), np.shape(train_target))
    print('Validation data size:', np.shape(valid_input),
          np.shape(valid_target))
    print('Test data size:', np.shape(tests_input), np.shape(tests_target))

    train_input_flatten = np.reshape(
        train_input, (len(train_input), (window_length)*len(hun_chars)))
    valid_input_flatten = np.reshape(
        valid_input, (len(valid_input), (window_length)*len(hun_chars)))
    tests_input_flatten = np.reshape(
        tests_input, (len(tests_input), (window_length)*len(hun_chars)))
    print('Network data generated successfully')

    return [train_input_flatten, train_target,
            valid_input_flatten, valid_target,
            tests_input_flatten, tests_target]
    

def train_data_generator_uchars(data_counter, window_length, length_after,
                                tag_chars='BM', tail_cut=100000,
                                valid_rate=0.2, test_rate=0.1):
    """Generate training data from counter data
        unique words -> characters -> unique -> randomize -> cut"""
    data_list = np.array(data_counter.most_common(tail_cut))[:,0]
    word_list, c_all, c_same_char_num = hyph_tupples(data_list,
                                                    tag_chars=tag_chars)
    print('Data read successfully')
    print('non-standard hyphenation:')
    print(c_same_char_num, c_all, c_same_char_num/c_all)

    # Generate network data
    data_in = []
    data_out = []
    wrong_word = 0
    data_in, data_out, wrong_word = tupple_to_train(word_list,
                                                    window_length,
                                                    length_after,
                                                    tag_chars=tag_chars)
    print('Data len: ', len(data_in))
    print('Words with unrecognized caracter: ', wrong_word)

    #Unique
    data_len = len(data_in)

    data_in = np.array(data_in, dtype='float32')
    data_out = np.array(data_out, dtype='float32')
    
    shape_in = np.shape(data_in)
    shape_out = np.shape(data_out)
    
    data_in_flatten = np.reshape(
        data_in, (shape_in[0], shape_in[1]*shape_in[2]))
    shape_in_flatten = np.shape(data_in_flatten)
    
    data_iosum = np.concatenate((data_in_flatten, data_out), axis=1)
    data_iosum_unique = np.vstack({tuple(row) for row in data_iosum})
    
    data_in = data_iosum_unique[:,:-shape_out[1]]
    data_out = data_iosum_unique[:,-shape_out[1]:]
    print('Data unique len: ', np.shape(data_iosum_unique)[0])
    
    data_len = len(data_in)
    data_in, data_out = unison_shuffled_copies(data_in, data_out)
    tests_input = data_in[0:int(data_len*test_rate)]
    tests_target = data_out[0:int(data_len*test_rate)]
    valid_input = data_in[int(data_len*test_rate):
                          int(data_len*(test_rate+valid_rate))]
    valid_target = data_out[int(data_len*test_rate):
                            int(data_len*(test_rate+valid_rate))]
    train_input = data_in[int(data_len*(test_rate+valid_rate)):]
    train_target = data_out[int(data_len*(test_rate+valid_rate)):]

    print('Training data size:', np.shape(train_input), np.shape(train_target))
    print('Validation data size:', np.shape(valid_input),
          np.shape(valid_target))
    print('Test data size:', np.shape(tests_input), np.shape(tests_target))

    train_input_flatten = np.reshape(
        train_input, (len(train_input), (window_length)*len(hun_chars)))
    valid_input_flatten = np.reshape(
        valid_input, (len(valid_input), (window_length)*len(hun_chars)))
    tests_input_flatten = np.reshape(
        tests_input, (len(tests_input), (window_length)*len(hun_chars)))
    print('Network data generated successfully')

    return [train_input_flatten, train_target,
            valid_input_flatten, valid_target,
            tests_input_flatten, tests_target]
    

In [5]:
def train_data_words(data_counter, tag_chars='BM', padding = 30, tail_cut=100000,
                     valid_rate=0.2, test_rate=0.1):
    """Training data, example: alma -> {[[1,0..][0,0..][0,0...][0,0...]],[[1,0],[0,1][1,0][0,1]]}"""
    data_list = np.array(data_counter.most_common(tail_cut))[:,0]
    word_list, c_all, c_same_char_num = hyph_tupples(data_list,
                                                    tag_chars=tag_chars)
    print('Data read successfully')
    print('non-standard hyphenation:')
    print(c_same_char_num, c_all, c_same_char_num/c_all)
    
    # Generate network data
    data_in = []
    data_out = []
    data_words = []
    wrong_word = 0
    long_word = 0
    for word in word_list:
        try:
            next_data_in, next_data_out = generate_network_words(word, padding = padding, tag_chars=tag_chars)
            next_data_in = np.array(next_data_in, dtype='float32')
            next_data_out = np.array(next_data_out, dtype='float32')
            data_in.append(next_data_in)
            data_out.append(next_data_out)
            data_words.append(word)
        except ValueError:
            wrong_word += 1
        except IndexError:
            long_word += 1
            
    print('Data len: ', len(data_in))
    print('Words with unrecognized caracter: ', wrong_word)
    print('Words longer than the padding: ', long_word)
    
    data_in = np.array(data_in)
    data_out = np.array(data_out)
    
    data_len = len(data_in)
    order = np.random.permutation(data_len)
    data_in = [data_in[k] for k in order]
    data_out = [data_out[k] for k in order]
    data_words = [data_words[k] for k in order]
    
    #data_in, data_out, word_list = unison_shuffled_copies(data_in, data_out, word_list)
    
    datas = {}
    
    datas["tests_words"] = np.array(data_words[0:int(data_len*test_rate)])
    datas["tests_input"] = np.array(data_in[0:int(data_len*test_rate)])
    datas["tests_target"] = np.array(data_out[0:int(data_len*test_rate)])
    datas["valid_words"] = np.array(data_words[int(data_len*test_rate):
                                               int(data_len*(test_rate+valid_rate))])
    datas["valid_input"] = np.array(data_in[int(data_len*test_rate):
                                            int(data_len*(test_rate+valid_rate))])
    datas["valid_target"] = np.array(data_out[int(data_len*test_rate):
                                              int(data_len*(test_rate+valid_rate))])
    datas["train_words"] = np.array(data_words[int(data_len*(test_rate+valid_rate)):])
    datas["train_input"] = np.array(data_in[int(data_len*(test_rate+valid_rate)):])
    datas["train_target"] = np.array(data_out[int(data_len*(test_rate+valid_rate)):])
    
    return datas, wrong_word, long_word

# Models

In [6]:
def model_creator_dnn(window_length, output_length, num_layers=1,
                  num_hidden=10, chars=hun_chars):
    """Creates Keras model with the given input, output dimensions
    and layer number, hidden layer length"""
    
    input_shape = window_length*len(chars)
    
    model = Sequential()
    
    model.add(Dense(input_dim=(input_shape),
                    units=num_hidden, name='input_layer',
                    activation='sigmoid'))
    for i in range(1, num_layers):
        model.add(Dense(units=num_hidden, activation='sigmoid'))

    # model.add(Flatten())
    model.add(Dense(output_length, name='output_layer', activation='softmax'))

    if(output_length == 2):
        model.compile(loss='binary_crossentropy', optimizer='adam')
    else:
        model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

def model_creator_cnn(output_length,kernel_size=10, strides=1, word_length = 30, chars=hun_chars):
    """Creates Keras CNN model"""

    model = Sequential()
    
    model.add(Conv1D(516,kernel_size, strides=strides, padding="same",
                     activation='relu', input_shape=(word_length, len(chars))))
    


    model.add(Dense((output_length), name = 'output_layer', activation='softmax'))
    
    if(output_length == 2):
        model.compile(loss='binary_crossentropy', optimizer='adam')
    else:
        model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

def model_creator_lstm(output_length, word_length = 30, chars = hun_chars):
    """Creates Keras LSTM model"""
    
    model = Sequential()
    
    model.add(LSTM(128, activation='relu',return_sequences=True,
                   go_backwards=True,
                   input_shape=(word_length, len(chars))))
    model.add(LSTM(64, activation='relu',return_sequences=True,
                   go_backwards=True,))
    
    model.add(Dense((output_length), name = 'output_layer', activation='softmax'))

    
    if(output_length == 2):
        model.compile(loss='binary_crossentropy', optimizer='adam')
    else:
        model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

# Main

In [7]:
padding = 30
tail_cut = 100000
window_length = 7
length_after = 3
tag_chars = 'BM'
num_layers = 5
num_hidden = 110

# Data read and network data generate
counter_hu_data = data_reader('web2.2-freq-sorted.txt',tail_cut, lang_selector=False)

datas, wrong_words, long_word = train_data_words(counter_hu_data, tag_chars,
                                                             padding, tail_cut)

tests_input_cnn = datas["tests_input"]
tests_target_cnn = datas["tests_target"]
valid_input_cnn = datas["valid_input"]
valid_target_cnn = datas["valid_target"]
train_input_cnn = datas["train_input"]
train_target_cnn = datas["train_target"]

tests_words = datas["tests_words"]
valid_words = datas["valid_words"]
train_words = datas["train_words"]


print(np.shape(train_input_cnn), np.shape(valid_input_cnn), np.shape(tests_input_cnn))

Data read successfully
non-standard hyphenation:
82563 83678 0.9866751117378523
Data len:  81910
Words with unrecognized caracter:  646
Words longer than the padding:  7
(57337, 30, 37) (16382, 30, 37) (8191, 30, 37)


In [14]:
wrong_word = 0
tests_input, tests_target, wrong_w_p = tupple_to_train(tests_words,
                                                        window_length,
                                                        length_after,
                                                        tag_chars=tag_chars)
wrong_word += wrong_w_p
valid_input, valid_target, wrong_w_p = tupple_to_train(valid_words,
                                                        window_length,
                                                        length_after,
                                                        tag_chars=tag_chars)
wrong_word += wrong_w_p
train_input, train_target, wrong_w_p = tupple_to_train(train_words,
                                                        window_length,
                                                        length_after,
                                                        tag_chars=tag_chars)
wrong_word += wrong_w_p

print("Still wrong word (expected zero): ", wrong_word)

train_input_flatten = np.reshape(
    train_input, (len(train_input), (window_length)*len(hun_chars)))
valid_input_flatten = np.reshape(
    valid_input, (len(valid_input), (window_length)*len(hun_chars)))
tests_input_flatten = np.reshape(
    tests_input, (len(tests_input), (window_length)*len(hun_chars)))

Still wrong word (expected zero):  0


In [8]:
tail_cut = 100000
window_length = 7
length_after = 3
tag_chars = 'BM'
num_layers = 5
num_hidden = 110

# Data read and network data generate
counter_hu_data = data_reader('web2.2-freq-sorted.txt',tail_cut, lang_selector=False)
[train_input_flatten, train_target,
 valid_input_flatten, valid_target,
 tests_input_flatten,
 tests_target] = train_data_generator(counter_hu_data,
                                             window_length,
                                             length_after,
                                             tag_chars,
                                             tail_cut)

#train_input_flatten_p1 = np.expand_dims(train_input_flatten, axis=1) # reshape (X, 1, 259) 
#valid_input_flatten_p1 = np.expand_dims(valid_input_flatten, axis=1)
#tests_input_flatten_p1 = np.expand_dims(tests_input_flatten, axis=1)

Data read successfully
non-standard hyphenation:
82563 83678 0.9866751117378523
Data len:  698309
Words with unrecognized caracter:  646
Training data size: (488817, 7, 37) (488817, 2)
Validation data size: (139662, 7, 37) (139662, 2)
Test data size: (69830, 7, 37) (69830, 2)
Network data generated successfully


In [9]:
# Creating the keras model
model_dnn = model_creator_dnn(window_length, len(tag_chars),
                      num_layers, num_hidden)

earlyStopping_dnn = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=20, verbose=0, mode='auto')

model_cnn = model_creator_cnn(len(tag_chars), kernel_size=15)

earlyStopping_cnn = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, verbose=0, mode='auto')

model_lstm = model_creator_lstm(len(tag_chars))

earlyStopping_lstm = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, verbose=0, mode='auto')

print('Models created. Start training')


Models created. Start training


In [40]:
from keras.utils import plot_model
plot_model(model_lstm, to_file='model.png', show_shapes=True)

In [31]:
history_lstm = model_lstm.fit(train_input_cnn, train_target_cnn,
                    epochs=1000, batch_size=1024,
                    validation_data=(valid_input_cnn, valid_target_cnn),
                    verbose=0, callbacks=[earlyStopping_lstm])

In [ ]:
history_cnn = model_cnn.fit(train_input_cnn, train_target_cnn,
                    epochs=1000, batch_size=1024,
                    validation_data=(valid_input_cnn, valid_target_cnn),
                    verbose=0, callbacks=[earlyStopping_cnn])

In [15]:
history_dnn = model_dnn.fit(train_input_flatten, train_target,
                    epochs=1000, batch_size=1024,
                    validation_data=(valid_input_flatten, valid_target),
                    verbose=0, callbacks=[earlyStopping_dnn])

In [16]:
# summarize history for loss
plt.figure(figsize=(13, 5))
plt.plot(history_cnn.history['loss'])
plt.plot(history_cnn.history['val_loss'])

plt.plot(history_lstm.history['loss'])
plt.plot(history_lstm.history['val_loss'])
plt.plot(history_dnn.history['loss'])
plt.plot(history_dnn.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['cnn_train', 'cnn_validation',
            'lstm_train', 'lstm_validation',
            'ffnn_train', 'ffnn_validation'], loc='upper right')
#plt.ylim((0.0, 0.02))
plt.grid()
plt.show()

NameError: name 'history_cnn' is not defined

In [38]:
model_dnn.save('models/mBMdnn3.h5')

In [51]:
model_dnn = keras.models.load_model('models/mBMdnnB.h5')
model_cnn = keras.models.load_model('models/mBMcnnB.h5')
model_lstm = keras.models.load_model('models/mBMlstmB.h5')

# Evaluation

In [49]:
def hyph_predict(word, model,
                 length=2, length_after=0, tag_chars='BMES', aslist=False, model_type='dnn'):
    """Generate tagging from the input word according to the model"""
    word_in = []
    word_out = []
    generate_network_data([word, len(word)*tag_chars[0]],
                          word_in, word_out, length=length,
                          length_after=length_after, tag_chars=tag_chars)
    word_in = np.reshape(word_in, (len(word_in), (length)*len(hun_chars)))
    if model_type=='cnn':
        word_in = np.expand_dims(word_in, axis=1) # reshape (x, 1, 259) 
    word_out = model.predict(word_in)
    tag_list = np.array(list(tag_chars))
    temp = np.argmax(word_out, axis=1)
    temp = tag_list[temp]
    if(aslist):
        return temp
    return "".join(temp)


def hyph_insterted(word, model,
                   length=2, length_after=0, tag_chars='BMES', model_type='dnn'):
    tags = hyph_predict(word, model,length,
                        length_after, tag_chars, aslist=False, model_type=model_type)
    word_inserted = ""
    if tag_chars=='BM':
        for i in range(len(word)):
            if i != 0 and tags[i]=='B':
                word_inserted += '-'
            word_inserted += word[i]
    else:
        raise NotImplementedError('BM implemented only')
    return word_inserted


def evaluation(wtags_predicted, wtags_target, tag_chars='BM'): 
    """Compare BMBM with BBMB"""
    if tag_chars!='BM':
        raise NotImplementedError("Only BM available")
    tp = 0 # target: B prediction: B
    tn = 0 # target: M prediction: M
    fp = 0 # target: M prediction: B
    fn = 0 # target: B prediction: M
    for i in range(min(len(wtags_target),len(wtags_predicted))):
        c_t = wtags_target[i]
        c_p = wtags_predicted[i]
        if (c_t == 'B') and (c_p == 'B'):
            tp +=1
        elif (c_t == 'M') and (c_p == 'M'):
            tn +=1
        elif (c_t == 'M') and (c_p == 'B'):
            fp +=1
        elif (c_t == 'B') and (c_p == 'M'):
            fn +=1
        else:
            raise ValueError("Not expected tag!" + c_t + c_p)
    good = False
    if fn+fp == 0:
        good = True
    return tp, tn, fp, fn, good


def test_ev(model, model_type, model_params = None, num_tests=-1, verbose=1,
            hypher=pyphen.Pyphen(lang='hu_HU')):
    """Evaulate the tests"""
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    good = 0
    if (verbose>0):
        print("Prediction\tTarget")
    if model_type == 'cnn' or model_type =='lstm':
        test_result = model.predict(tests_input_cnn[0:num_tests])
        for i in range(len(test_result)):
            test_tags = result_decode(test_result[i])
            test_word = tests_words[i,0]
            ev = evaluation(test_tags, tests_words[i,1])
            tp += ev[0]
            tn += ev[1]
            fp += ev[2]
            fn += ev[3]
            if ev[4]:
                good+=1
            
            if (verbose>0) and (ev[4] == False):
                print(hyp_inserted(test_word,test_tags),'\t',
                      hypher.inserted(test_word))
                

    if model_type == 'dnn':
        test_range = num_tests
        if num_tests==-1:
            test_range = len(tests_words)
        window_length = model_params["window_length"]
        length_after = model_params["length_after"]
        tag_chars = model_params["tag_chars"]
        for i in range(test_range):
            test_word = tests_words[i,0]
            test_tags = hyph_predict(test_word, model_dnn,
                                       window_length, length_after,
                                       tag_chars,
                                       model_type=model_type)
            ev = evaluation(test_tags, tests_words[i,1])
            tp += ev[0]
            tn += ev[1]
            fp += ev[2]
            fn += ev[3]
            if ev[4]:
                good+=1
            
            if (verbose>0) and (ev[4] == False):
                print(hyp_inserted(test_word,test_tags),'\t',
                      hypher.inserted(test_word))
    
    test_precision = tp / (tp + fp)
    test_recall = tp / (tp + fn)
    test_Fscore = 2 * (test_precision *
                       test_recall) / (test_precision + test_recall)
    ret = {}
    ret["precision"] = test_precision
    ret["recall"] = test_recall
    ret["Fscore"] = test_Fscore
    ret["word_rate"] = good/len(tests_words)
    return ret
        
    
def result_decode(result, tag_chars='BM'):
    """[[0,1][0,1]] -> 'MM'"""
    tags = ""
    result = hardmax(result)
    for c in result:
        tags += one_hot_decode(c, tag_chars)
    return tags


def hardmax(arr, axis = -1):
    """Return 1 if the value is the max in the row, 0 otherwise
    [0.2,0.4,0.5]->[0,0,1]"""
    temp = arr - np.max(arr, axis=axis, keepdims=True)
    return np.round(1+temp)


def hyp_inserted(word, tags, tag_chars='BM'):
    """insert hyphen to the tags"""
    assert len(word)<=len(tags)
    s = ""
    for c in range(len(word)):
        if (c!=0) and tags[c]=='B':
            s+='-'
        s+=word[c]
    return s

In [52]:
#print("Prediction \t Target")

print("Evaluation on",len(tests_words),"words.")

model_params = {}
model_params["window_length"] = window_length
model_params["length_after"] = length_after
model_params["tag_chars"] = tag_chars

print("CNN:",test_ev(model_cnn,"cnn", verbose=0))
print("LSTM:",test_ev(model_lstm,"lstm", verbose=0))
print("FFNN:",test_ev(model_dnn,"dnn",model_params=model_params, verbose=0))

Evaluation on 8191 words.
CNN: {'precision': 0.9877300613496932, 'word_rate': 0.9625198388475156, 'recall': 0.9970382337102854, 'Fscore': 0.9923623207825272}
LSTM: {'precision': 0.9817491937013849, 'word_rate': 0.9479916982053473, 'recall': 0.9952304023386415, 'Fscore': 0.9884438332091763}
FFNN: {'precision': 0.9882924589662164, 'word_rate': 0.9554388963496521, 'recall': 0.9935, 'Fscore': 0.9908893875750426}


In [41]:
model_type = "dnn"

test_tp = 0
test_fp = 0
test_tn = 0
test_fn = 0
test_str = ""

if model_type == "dnn":
    test_results = model_dnn.predict(tests_input_flatten)
    history = history_dnn
else:
    if model_type == "cnn":
        test_results = model_cnn.predict(tests_input_flatten_p1)
        history = history_cnn
if(np.shape(test_results)[1] == 2):
    for i in range(len(test_results)):
        # positive
        if np.argmax(test_results[i]) == 1:
            if np.argmax(tests_target[i]) == 1:
                test_tn += 1
            else:
                test_fn += 1
        else:
            if np.argmax(tests_target[i]) == 1:
                test_fp += 1
            else:
                test_tp += 1
    print(np.shape(test_results),np.shape(tests_target))
    print(test_tp,test_fp,test_tn,test_fn)
    test_precision = test_tp / (test_tp + test_fp)
    test_recall = test_tp / (test_tp + test_fn)
    test_Fscore = 2 * (test_precision * 
                       test_recall) / (test_precision + test_recall)
    test_str = str(test_precision) + '\t' + str(test_recall) 
    test_str += '\t' + str(test_Fscore)
else:
    for i in range(len(test_results)):
        if np.argmax(test_results[i]) == np.argmax(tests_target[i]):
            test_success += 1
        else:
            test_fail += 1
    test_str = str(test_fail/(test_fail+test_success))

with open("results_data_types.txt", "a") as myfile:
    result = ""
#    result += str(window_length) + '\t'
#    result += str(length_after) + '\t' + tag_chars
#    result += '\t' + str(num_layers) + '\t'
#    result += str(num_hidden) + '\t'
    result += model_type + '\t'
    result += str(history.epoch[-1]) + '\t'
    result += str(history.history['val_loss'][-1])
    result += '\t' + test_str
    result += '\n'
    myfile.write(result)

(69745, 2) (69745, 2)
25709 281 43464 291


## Words

In [15]:
hypher = pyphen.Pyphen(lang='hu_HU')

In [ ]:
test = 'szemüveg'
print('Word:', test, 'Prediction:',
      hyph_predict(test, model, window_length, length_after, tag_chars, model_type='cnn'),
      'Target:', hyph_tags_4to2(hyph_tags(test)), hypher.inserted(test))

test = 'leopárd'
print('Word:', test, 'Prediction:',
      hyph_predict(test, model, window_length, length_after, tag_chars, model_type='cnn'),
      'Target:', hyph_tags_4to2(hyph_tags(test)), hypher.inserted(test))

In [ ]:
test_words = counter_hu_data.most_common()[-400:]
print('Prediction\tTarget')
for word in test_words:
    next_word = word[0]
    if(len(next_word) != 0 and same_char_num(next_word)):
        try:
            predicted_value = hyph_predict(next_word, model,
                                           window_length, length_after,
                                           tag_chars, model_type='cnn')
            predicted_visual = hyph_insterted(next_word, model,
                                              window_length, length_after,
                                              tag_chars, model_type='cnn')
            excepted_value = hyph_tags_4to2(hyph_tags(next_word))
            success = predicted_value == excepted_value
            if not success:
                print(predicted_visual,
                        '\t',hypher.inserted(next_word))
        except ValueError as e:
            print(str(e))

# Sandbox

In [38]:
hypher = pyphen.Pyphen(lang='hu_HU')

N = -1
print("FFNN\tCNN\tLSTM\tTarget")
for i in range(len(tests_words[0:N])):
    hyphs = ""
    test_word = tests_words[i,0]
    test_ic = tests_input_cnn[i:i+1]
    test_tags = hyph_predict(test_word, model_dnn,
                             window_length, length_after,
                             tag_chars,
                             model_type="dnn")
    ffnn = hyp_inserted(test_word,test_tags)
    hyphs +=ffnn+'\t'
    
    test_result = model_cnn.predict(test_ic)
    test_result = test_result[0]
    test_tags = result_decode(test_result)
    
    cnn = hyp_inserted(test_word,test_tags)
    hyphs +=cnn+'\t'
    
    test_result = model_lstm.predict(test_ic)
    test_result = test_result[0]
    test_tags = result_decode(test_result)
    
    lstm = hyp_inserted(test_word,test_tags)
    hyphs +=lstm+'\t'
    
    target = hypher.inserted(test_word) 
    hyphs += target
    if (ffnn!=target) or (cnn!=target) or (lstm!=target):
        print(hyphs)

FFNN	CNN	LSTM	Target
pro-mo-ting	promo-ting	pro-mo-ting	promo-ting
köz-út	köz-út	kö-zút	köz-út
tan-évet	ta-n-é-vet	ta-n-é-vet	tan-évet
de-ci-si-on-mak-ing	de-ci-si-on-mak-ing	de-ci-si-on-ma-k-ing	de-ci-si-on-mak-ing
fi-deszf-rak-ció	fi-de-szf-rak-ció	fi-deszfrak-ció	fi-desz-frak-ció
weeks	we-eks	we-eks	weeks
libg-no-me-can-vas	libg-no-me-can-vas	libg-no-m-e-c-an-vas	libg-no-mec-an-vas
beson-ders	beson-ders	be-son-ders	beson-ders
strengt-he-ned	strengt-he-ned	strengt-he-n-ed	strengt-he-ned
goods	goods	go-ods	goods
ké-p-ü-ze-net	ké-p-ü-ze-net	kép-üze-net	kép-üze-net
dist-ri-bu-ci-ón	dis-t-ri-bu-ci-ón	dist-ri-bu-ci-ón	dis-t-ri-bu-ci-ón
fi-ve	fi-ve	fi-ve	five
szpon-zo-ri	szpon-zo-ri	sz-pon-zo-ri	szpon-zo-ri
el-sö-sor-ban	el-sö-sor-ban	el-sö-sor-ban	el-sö-s-or-ban
ch-ris-ti-na	ch-ris-tina	ch-ris-ti-na	ch-ris-tina
un-ga-ris-che	un-ga-ris-c-he	un-ga-ris-che	un-ga-ris-che
gyászt-va-ló-já-ban	gyásztva-ló-já-ban	gyászt-va-ló-já-ban	gyásztva-ló-já-ban
in-ter-ak-ci-ók	in-te-r-ak-ci-ók	in-ter-ak-ci

In [27]:
one_hot_encode(hardmax(test_result))

TypeError: 'in <string>' requires string as left operand, not numpy.ndarray

In [66]:
np.random.random(5)

array([ 0.61187395,  0.66373782,  0.82366587,  0.89691012,  0.51733389])